In [1]:
# dependencies, already included with anaconda
import os                                     # file manipulation
import re                                     # text comprehension
import time                                   # execution time analysis
from datetime import date, datetime           # time and date to string
from PIL import Image, ImageOps, ImageEnhance # image manipulation

# dependencies, install with the following shell command
# $ conda install -c conda-forge <package>
import pyautogui      # keyboard/mouse/screenshot manipulation
import pytesseract    # python wrapper for ocr
# to use pytesseract, you must also install tesseract-ocr
# see https://github.com/UB-Mannheim/tesseract/wiki, for installation instructions
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [2]:
# working with the mouse
def print_pos():
    x, y = pyautogui.position() # locate the cursor
    print(str(x) + ", " + str(y))
# pyautogui.moveTo(787,72)    # moving the cursor
# pyautogui.doubleClick()     # double-click
# pyautogui.rightClick()    # right-click

# working with the keyboard

# working with screen capture
# img = pyautogui.screenshot()

# close live help window
# pyautogui.click(1695,689)

In [9]:
def open_file():
    filename = "realdata-" + str(date.today()) + ".tsv"
    # filename  = "test.tsv"
    csvheader = "time\tsymbol\tbid\task\topen\thigh\tlow\tclose\tvolume\n"
    try:
        csvfile = open(filename, 'r')
        if csvfile.readline() != csvheader:
            print("malformed csv, resetting file")
            csvfile.close()
            os.remove(filename)
            csvfile = open(filename, 'x')
            csvfile.write(csvheader)
        csvfile.close()
    except FileNotFoundError:
        csvfile = open(filename, 'x')
        csvfile.write(csvheader)
    csvfile = open(filename, 'a')
    print("file open")
    return csvfile
    
# start timer
def start_timer():
    t_start    = time.time()
    t_startsec = time.localtime().tm_sec
    return (t_start, t_startsec)
    
def screen_cap_and_extract(debug_level = 1):
    # screen cap and extract
    #     0. screen capture and check trading hour
    img = pyautogui.screenshot()
    # img = Image.open("market_open.PNG")
    box = (2486, 70, 2700, 104)
    sub_img = img.crop(box)
    sub_img = ImageOps.invert(sub_img)
    img_enh = ImageEnhance.Contrast(sub_img)
    sub_img = img_enh.enhance(5.0)
    img_enh = ImageEnhance.Color(sub_img)
    sub_img = img_enh.enhance(0.0)
    hours_text = pytesseract.image_to_string(sub_img)
    current_time = re.search("[0-9]+:[0-9]+:[0-9]+", hours_text).group(0)
    offset = 0 if ("Market Open" in hours_text) else 27
    if debug_level > 2: sub_img.show()
    if debug_level > 1: print(hours_text)
    
    #     1. capture and extract data
    extract_pattern     = "[0-9]+[\.,][0-9]+"
    #     1a. capture symbol:
    box = (21, 257, 131, 285)
    sub_img = img.crop(box)
    symbol  = pytesseract.image_to_string(sub_img)
    #     1a. capture bid and ask: 
    box = (39, 298 + offset, 465, 326 + offset)
    sub_img = img.crop(box)
    img_enh = ImageEnhance.Contrast(sub_img)
    sub_img = img_enh.enhance(5.0)
    img_enh = ImageEnhance.Color(sub_img)
    sub_img = img_enh.enhance(0.0)
    bid_and_ask_text    = pytesseract.image_to_string(sub_img)
    bid_and_ask_pattern = "Bid: [0-9]+\.[0-9]+ Ask: [0-9]+\.[0-9]+"
    bid_and_ask         = None if re.match(bid_and_ask_pattern, bid_and_ask_text) is None \
                               else re.findall(extract_pattern, bid_and_ask_text)
    if debug_level > 2: sub_img.show()
    if debug_level > 1: print(bid_and_ask_text)
    
    #     1b. capture candlestick
    # box = (12, 1089 + offset, 696, 1108 + offset)
    box = (171, 1086 + offset, 696, 1110 + offset)
    sub_img = img.crop(box)
    img_enh = ImageEnhance.Contrast(sub_img)
    sub_img = img_enh.enhance(5.0)
    img_enh = ImageEnhance.Color(sub_img)
    sub_img = img_enh.enhance(0.0)
    candle_text    = pytesseract.image_to_string(sub_img)
    candle_pattern = "Open: [0-9]+\.[0-9]+ High: [0-9]+\.[0-9]+ Low: [0-9]+\.[0-9]+ Close: [0-9]+\.[0-9]+ Volume: [0-9]+,[0-9]+"
    candle         = None if re.match(candle_pattern, candle_text) is None \
                          else re.findall(extract_pattern, candle_text)
    if debug_level > 2: sub_img.show()
    if debug_level > 1: print(candle_text)
    
    #     1c. capture level 2's
    #     row height: 20 px
    # return data
    if bid_and_ask is None or candle is None:
        return (current_time, None)
    else:
        return (current_time, [symbol] + bid_and_ask + candle)

# log data
def log_data(write_obj, current_time, data_list, delimiter = '\t', debug_level = 1):
    if data_list is None:
        if debug_level > 0: print(current_time + ": bad data")
    else:
        row = current_time
        for item in data_list:
            row = row + delimiter + str(item)
        write_obj.write(row + '\n')
        if debug_level > 1: print(row)
    
# process screen cap
# 1. capture necessary information
# 2. label previous data

# perform actions, if necessary

# poll timer until d_time has elapsed

def poll_to_end_of_sec(t_startsec):
    t_stopsec = time.localtime().tm_sec
    while(t_startsec == t_stopsec):
        t_stopsec = time.localtime().tm_sec
        
def show_stats(t_start):  
    t_stop = time.time()
    print("Elapsed time: ", t_stop - t_start)
    
def loop_me(write_obj):
    t_start, t_startsec = start_timer()
    current_time, data_list = screen_cap_and_extract()
    log_data(write_obj, current_time, data_list)
    show_stats(t_start)
    poll_to_end_of_sec(t_startsec)


write_obj = open_file()
try:
    while True:
        loop_me(write_obj)
except KeyboardInterrupt:
    write_obj.close()
    print("collection terminated")

file open
17:42:02: bad data
Elapsed time:  0.9612255096435547
17:42:03: bad data
Elapsed time:  0.9021272659301758
17:42:04: bad data
Elapsed time:  0.9136745929718018
17:42:05: bad data
Elapsed time:  0.971893310546875
17:42:06: bad data
Elapsed time:  1.0725774765014648
Elapsed time:  0.9373352527618408
Elapsed time:  0.9548683166503906
Elapsed time:  0.9306964874267578
Elapsed time:  0.9555296897888184
Elapsed time:  0.9804074764251709
Elapsed time:  0.988943338394165
Elapsed time:  0.9612572193145752
Elapsed time:  1.0112626552581787
Elapsed time:  0.9788825511932373
Elapsed time:  1.0046181678771973
Elapsed time:  1.07122802734375
Elapsed time:  0.9832255840301514
collection terminated


In [ ]:
# use a limit order to enter, a stop order to stop loss
# update the stop loss
# rule of thumb, target is twice the risk
# risk level is the stop loss
# example buy
#    entry:  $20.00/share
#    stop:   $19.80/share
#    risk:  $200.00 at 1000 shares
# typical batting average: 50%
